# CAP 6640 
# Script Assignment 1
# Jan 25, 2024

# Group 4
# Andres Graterol
#                   UCF ID: 4031393
# Zachary Lyons
#                   UCF ID: 4226832
# Christopher Hinkle
#                   UCF ID: 4038573
# Nicolas Leocadio
#                   UCF ID: 3791733

We start by reading our corpus data, which is a list of tech news articles found from kaggle

https://www.kaggle.com/datasets/harshityadav95/tech-news-articles

There is a requirements.txt file with all the needed python modules, "pip install -r requirements.txt"

Old: Prior to running, download corpus from top right of link and then extract to a folder called "archive" in the same directory as this file
New: Extract archive.zip (its 1.5GB lol) to ./archive


------------- ./ScriptAssignment2.ipynb

------------- ./archive/CSV_sampleDataset_DataAnalyticsTrack.csv

In [3]:
import nltk
import re
nltk.download('punkt')  # one time execution if not cached yet
nltk.download('stopwords')  # one time execution if not cached yet

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nick_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nick_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def read_data(file_path):
    # labels = []
    corpus = []

    with open(file_path, encoding="utf8", errors='ignore') as f_input:
        corpus.append((f_input.read()))
        # labels.append(file_path.split('/')[-2])
    return corpus

corpus = read_data("archive/CSV_sampleDataset_DataAnalyticsTrack.csv")

In [5]:
def tokenize(corpus):
    regex = re.compile('[^a-zA-Z0-9]')  # regex to only match alphanumeric characters
    tokens = []
    
    for sent in nltk.sent_tokenize(corpus):
        for word in nltk.word_tokenize(sent):
            clean_word = regex.sub('', word)
            if clean_word:  # make sure we're not appending empty strings
                tokens.append(clean_word.lower())
    return tokens

tokens = tokenize(corpus[0])
tokens[50:60]

['becoming',
 'increasingly',
 'difficult',
 'particularly',
 'as',
 'embedded',
 'systems',
 'become',
 'more',
 'specialized']

In [6]:
def remove_stopwords(tokens):
    stopwords = nltk.corpus.stopwords.words('english')
    return [word for word in tokens if word not in stopwords]

no_stopwords = remove_stopwords(tokens)
no_stopwords[50:60]

['jose',
 'calif',
 'lexra',
 'inc',
 'san',
 'jose',
 'resolves',
 'many',
 'problems',
 'configurable']

In [7]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def stem_words(word):
    return stemmer.stem(word).strip()

stemmed = [stem_words(word) for word in no_stopwords]
stemmed[50:60]

['jose',
 'calif',
 'lexra',
 'inc',
 'san',
 'jose',
 'resolv',
 'mani',
 'problem',
 'configur']

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.8, max_features=200000,
                        min_df=0.05, stop_words='english',
                        use_idf=True, tokenizer=tokenize,
                        preprocessor=stem_words, lowercase=True)
tfidf_matrix = tfidf.fit_transform(corpus)

c:\Users\nick_\Documents\MSCpE\CAP6640 Natural Language Processing\venv\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\nick_\Documents\MSCpE\CAP6640 Natural Language Processing\venv\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', '